In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from standard_function import *
from return_detail_icd_code import *
import pickle
import unicodedata

In [2]:
df = pd.read_excel("ICD_new.xlsx", header = None, encoding = 'utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10126 entries, 0 to 10125
Data columns (total 3 columns):
0    10126 non-null object
1    10126 non-null object
2    10126 non-null object
dtypes: object(3)
memory usage: 237.4+ KB


In [3]:
df1=''
for i in range(0, df.shape[0]):
    df1 += df[0][i][0:3]+ " "

In [4]:
def Convert(string): 
    li = list(string.split(" ")) 
    return li 
       
df2 = Convert(df1)

In [5]:
df[3] = pd.Series(df2)
df.head()

0     1                                                  2    3
0   A00  DG19                                            Bệnh tả  A00
1  A000  DG19  Bệnh tả do Vibrio cholerae 01, typ sinh học ch...  A00
2  A001  DG19  Bệnh tả do Vibrio cholerae 01, typ sinh học eltor  A00
3  A009  DG19                            Bệnh tả, không đặc hiệu  A00
4   A01  DG19                  Bệnh thương hàn và phó thương hàn  A01

In [6]:
data_frame = df[[0,2,3]]
data_frame.head(20)

0                                                  2    3
0    A00                                            Bệnh tả  A00
1   A000  Bệnh tả do Vibrio cholerae 01, typ sinh học ch...  A00
2   A001  Bệnh tả do Vibrio cholerae 01, typ sinh học eltor  A00
3   A009                            Bệnh tả, không đặc hiệu  A00
4    A01                  Bệnh thương hàn và phó thương hàn  A01
5   A010                                         Thương hàn  A01
6   A011                              Bệnh phó thương hàn A  A01
7   A012                              Bệnh phó thương hàn B  A01
8   A013                              Bệnh phó thương hàn C  A01
9   A014                Bệnh phó thương hàn, không đặc hiệu  A01
10   A02                              Nhiễm salmonella khác  A02
11  A020                            Viêm ruột do Salmonella  A02
12  A021                    Nhiễm trùng huyết do Salmonella  A02
13  A022                           Nhiễm salmonella khu trú  A02
14  A028               Nhiễm trùng salmonella xác định khác  A02
15  A029        Nhiễm trùng salmonella, không đặc hiệu khác  A02
16   A03                                 Bệnh lỵ trực khuẩn  A03
17  A030         Bệnh lỵ trực khuẩn do Shigella dysenteriae  A03
18  A031            Bệnh lỵ trực khuẩn do Shigella flexneri  A03
19  A032              Bệnh lỵ trực khuẩn do Shigella boydii  A03

In [7]:
target = data_frame[3]
type(target)

pandas.core.series.Series

In [8]:
data_frame[2] = data_frame[2].apply(delete_special_char)
#data_frame[2] = data_frame[2].apply(un_punctuation)
data_frame[2][4414]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


'phì đại amyđan'

In [9]:
data_frame[2][1][1]

'ệ'

In [10]:
feature=data_frame[2]
feature[:5]

0                                              bệnh tả
1    bệnh tả do vibrio cholerae 01  typ sinh học ch...
2    bệnh tả do vibrio cholerae 01  typ sinh học eltor
3                              bệnh tả  không đặc hiệu
4                    bệnh thương hàn và phó thương hàn
Name: 2, dtype: object

In [11]:
text_data = np.array(feature)
text_data[600]

'bệnh hiv dẫn đến rối loạn huyết học và miễn dịch  chưa phân loại'

In [12]:
with open('sample_vector', 'wb') as fp:
    pickle.dump(text_data, fp)
print("Save Sample ")

Save Sample 


In [13]:
count = CountVectorizer(decode_error='ignore')
count.fit(text_data)
print(count)
bag_of_words = count.transform(text_data)
bag_of_words

CountVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


<10126x2425 sparse matrix of type '<class 'numpy.int64'>'
	with 78033 stored elements in Compressed Sparse Row format>

In [14]:
X = bag_of_words.toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
y = np.array(target)
y

array(['A00', 'A00', 'A00', ..., 'Y84', 'Z00', '001'], dtype=object)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,random_state = 1)

In [17]:
params = {'alpha':[0.001,0.003,0.005,0.007,0.01,0.015]}
clf = BernoulliNB()

clf= GridSearchCV(clf, params, cv = 5)
clf.fit(X_train, y_train)
print(clf.best_params_)

best_clf = clf.best_estimator_
y_pred = best_clf.predict(X_test)

print(" training Size = %d, accuracy = %.2f%%" % \
     (len(X_train), accuracy_score(y_test,y_pred ) *100))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'alpha': 0.007}
 training Size = 9113, accuracy = 62.09%


In [18]:
f = open('Eclaim_predict.pickle', 'wb')
pickle.dump(clf, f)
f.close()
print("Model Saved..")

Model Saved..


In [19]:
loaded_model = pickle.load(open('Eclaim_predict.pickle', 'rb'))
#result = loaded_model.predict(y_test)
#print(result)

In [22]:
data = ["tắc tá tràng"]
y_test = count.transform(data)
result = loaded_model.predict(y_test)
result

array(['K31'], dtype='<U3')

In [23]:
target = data_frame[3]
data_frame[2] = data_frame[2].apply(delete_special_char)
#data_frame[2] = data_frame[2].apply(un_punctuation)
data_frame[2][4413]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


'phì đại va1'